# Notebook explanation

This notebook is designed to process and analyze orders using Databricks and Apache Spark.

## 1. Data intake in streaming (Bronze Layer)

The `bronze_stream` function is defined to read data in JSON format from an landing zone (Landing Zone) using Spark Structured Streaming. The data scheme includes geographic information, date, customer and employee identifiers, number of products and the order identifier.

- ** Duplicate elimination: ** Duplicates are eliminated based on `Order_id`.
- ** Writing in Delta Lake: ** Clean data is written in a delta table, allowing efficient consultations and handling large data volumes.
- ** Checkpointing: ** A CheckPoint location is used for failure tolerance and stream secure reset.

## 2. Stream Execution

The `bronze_stream` function is executed by passing the corresponding routes and table names to initiate the intake and storage process of orders.

---

This flow allows you to analyze feelings of reviews and process orders in real time, facilitating data -based decision making and the integration of advanced analysis into business data pipelines.
""

With Open ("/workspace/explanation_notebook.md", "w", encoding = "utf-8") as f:
    F.Write (Notebook_Explanation)

In [0]:
%run ../Transversal/config

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType


def bronze_stream(landing_zone, checkpoint, table):

    schema = StructType() \
        .add("latitude", DoubleType()) \
        .add("longitude", DoubleType()) \
        .add("date", StringType()) \
        .add("customer_id", StringType()) \
        .add("employee_id", StringType()) \
        .add("quantity_products", IntegerType()) \
        .add("order_id", StringType())

    raw_stream_df = (
        spark.readStream
            .format("json")
            .schema(schema)
            .option("path", landing_zone)
            .load()
    )

    clean_stream_df = raw_stream_df.dropDuplicates(["order_id"])  # It must be changed to Watermark and Window when they are many records.

    query = (
        clean_stream_df.writeStream
            .format("delta")
            .outputMode("append")
            .option("checkpointLocation", checkpoint)
            .trigger(once=True)
            .option("mergeSchema", "true")
            .toTable(table))

    return query


query = bronze_stream(
            landing_zone=landing_zone_path,
            checkpoint=checkpoint_bronze,
            table=Bronze_Orders)